In [1]:
import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.sparse as sp
import gseapy as gp

In [2]:
import torch, numpy as np, random
seed = 1234
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)


In [3]:
import scanpy as sc

ST = sc.read_h5ad('/data/keerthana/integration/scMODAL/xenium_breast_cancer.h5ad')
ST.var_names_make_unique()
SC = sc.read_h5ad('/data/keerthana/integration/scMODAL/xenium_sc_data.h5ad')
SC.var_names_make_unique()

/home/keerthana/anaconda3/envs/scmodal_scviva/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [4]:
import pandas as pd

# Define mapping from fine-grained ST types to major SC types
mapping = {
    # Lymphoid
    'B_Cells': 'B-cells',
    'CD4+_T_Cells': 'T-cells',
    'CD8+_T_Cells': 'T-cells',
    'Stromal_&_T_Cell_Hybrid': 'T-cells',
    'T_Cell_&_Tumor_Hybrid': 'T-cells',

    # Myeloid
    'Macrophages_1': 'Myeloid',
    'Macrophages_2': 'Myeloid',
    'Mast_Cells': 'Myeloid',
    'IRF7+_DCs': 'Myeloid',
    'LAMP3+_DCs': 'Myeloid',
    'TFR+_DCs': 'Myeloid',

    # Epithelial
    'Invasive_Tumor': 'Cancer Epithelial',
    'Prolif_Invasive_Tumor': 'Cancer Epithelial',
    'DCIS_1': 'Cancer Epithelial',
    'DCIS_2': 'Cancer Epithelial',
    'Myoepi_ACTA2+': 'Normal Epithelial',
    'Myoepi_KRT15+': 'Normal Epithelial',

    # Endothelial
    'Endothelial': 'Endothelial',
    'Perivascular-Like': 'Endothelial',

    # Stromal
    'Stromal': 'CAFs',

    # Unlabeled / others
    'Unlabeled': 'Unlabeled'
}

# Apply mapping
ST.obs['celltype_major'] = ST.obs['cell_type'].map(mapping)

# If any fine types weren't mapped
unmapped = ST.obs.loc[ST.obs['celltype_major'].isna(), 'cell_type'].unique()
if len(unmapped) > 0:
    print("Unmapped cell types:", unmapped)
else:
    print("✅ All cell types mapped successfully!")



✅ All cell types mapped successfully!


if "counts" not in ST.layers:
    ST.layers["counts"] = ST.X.copy()
# make sure sample is a category
ST.obs["sample"] = ST.obs.get("sample", "sample_1")
ST.obs["sample"] = ST.obs["sample"].astype(str).astype("category")


In [5]:
# Reorder ST and SC so that common columns are first and in the same order
shared_genes = [g for g in ST.var_names if g in SC.var_names]  # preserves ST order
ST = ST[:, shared_genes]
SC = SC[:, shared_genes]

# keep track of common_vars length
common_vars = shared_genes
shared_gene_num = len(common_vars)



In [6]:
import scmodal
model = scmodal.model.Model(training_steps=100)
model.preprocess(ST,SC, shared_gene_num= shared_gene_num,alpha = None, eta = None)

preprocess called with alpha: None and eta: None
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [7]:
model.train()

Begining time:  Thu Nov  6 22:01:41 2025
step 0, loss_D=0.061273, loss_GAN=-0.052620, loss_AE=453.772888, loss_Geo=-14.519677, loss_LA=2799.227295, loss_MNN=74.110893
Ending time:  Thu Nov  6 22:01:48 2025
Training takes 7.43 seconds


In [8]:
model.eval()

Begining time:  Thu Nov  6 22:01:48 2025
Ending time:  Thu Nov  6 22:01:48 2025
Evaluating takes 0.14 seconds


Ending time:  Thu Nov  6 11:22:57 2025
Evaluating takes 0.14 seconds


In [9]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd

def impute_cell_types(model, ST, SC, n_neighbors=10):
    emb_A = model.latent[:ST.shape[0], :]
    emb_B = model.latent[ST.shape[0]:, :]

    nbrs = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine').fit(emb_B)
    knn_idx = nbrs.kneighbors(emb_A, return_distance=False)

    # Factorize string labels to numeric
    labels = np.array(SC.obs["celltype_major"].values)
    codes, uniques = pd.factorize(labels)
    neighbor_codes = codes[knn_idx]

    # Majority vote using bincount (fast + numeric)
    majority_idx = np.apply_along_axis(lambda x: np.bincount(x).argmax(), 1, neighbor_codes)
    majority_labels = uniques[majority_idx]

    ST.obs["imputed_cell_type"] = majority_labels
    return ST


In [10]:
ST = impute_cell_types(model, ST, SC, n_neighbors=10)
print(len(np.unique(ST.obs['imputed_cell_type'])))
from sklearn.metrics import adjusted_rand_score
ari = adjusted_rand_score(ST.obs['celltype_major'], ST.obs['imputed_cell_type'])
ari

/tmp/ipykernel_2029884/3903814323.py:21: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ST.obs["imputed_cell_type"] = majority_labels


9


0.5711575945594976

In [11]:
#actual loop starts here - now that we have first guess of cell types

In [12]:
# --- ensure counts layer is present and sample is categorical string ---
if "counts" not in ST.layers:
    ST.layers["counts"] = ST.X.copy()
# make sure sample is a category
ST.obs["sample"] = ST.obs.get("sample", "sample_1")
ST.obs["sample"] = ST.obs["sample"].astype(str).astype("category")

In [13]:
import scvi
import scanpy as sc
from scvi.external import SCVIVA
import logging

scvi.settings.seed = 42
scvi.settings.verbosity = logging.CRITICAL

Seed set to 42


In [14]:
n_iter = 9

In [15]:
aris = []
for i in range(n_iter):
    print(f"🔄 Iteration {i+1}/{n_iter}")

    # 5. SCANVI initialization and training
    scvi.model.SCVI.setup_anndata(
        ST,
        layer="counts",            # raw counts for modeling
        batch_key="sample",        # if you have one batch, set to a constant
        labels_key="imputed_cell_type",    # for SCANVI later
    )

    # 4. Train SCVI
    vae = scvi.model.SCVI(ST)
    vae.train(max_epochs=100)

    scanvi_model = scvi.model.SCANVI.from_scvi_model(vae, unlabeled_category="Unknown")
    scanvi_model.train(max_epochs=50)

    ST.obsm["X_scANVI"] = scanvi_model.get_latent_representation()

    setup_kwargs = {
            # "sample_key": "sample",  # column in adata.obs that contains the individual slide ID
            "labels_key": "imputed_cell_type",  # column in adata.obs that contains the cell type labels
            "cell_coordinates_key": "spatial",  # spatial coordinates key in adata.obsm
            "expression_embedding_key": "X_scANVI",  # expression embedding key in adata.obsm
            "sample_key": "sample"
        }
    scvi.external.SCVIVA.preprocessing_anndata(
        ST,
        k_nn=20,  # number of nearest neighbors for spatial graph construction
        **setup_kwargs,
    )
    scvi.external.SCVIVA.setup_anndata(
        ST,
        layer="counts",  # adata layer that contains the raw counts
        batch_key="sample",  # column in adata.obs that contains the batch covariate
        **setup_kwargs,
    )

    nichevae = scvi.external.SCVIVA(ST, gene_likelihood="zinb")

    nichevae.train(
        max_epochs=50,
        early_stopping=True,
        batch_size=min(512, ST.shape[0]),
        plan_kwargs={
            "lr": 5e-4,
        },
    )

    ST.obsm["X_scVIVA"] = nichevae.get_latent_representation()
    #add rownames to ST.obsm[X_scVIVA]
    ST.obsm["X_scVIVA"] = pd.DataFrame(ST.obsm["X_scVIVA"], index=ST.obs.index)


    alpha = ST.obsm['niche_composition']
    eta = ST.obsm["X_scVIVA"]

    model.update_embeddings(alpha=alpha, eta=eta)
    if i == 0:
        model.train(load_ckpt=False)  # Start from scratch
    else:
        model.train(load_ckpt=True)  # Continue from saved weights
    model.eval()

    ST = impute_cell_types(model, ST, SC, n_neighbors=10)

    #keep only first word

    ari = adjusted_rand_score(ST.obs['celltype_major'], ST.obs['imputed_cell_type'])
    aris.append(ari)
    print(f"Iteration ARI: {ari}")

    # if i > 0 and abs(ari - prev_ari) < 0.005:
    #     print("Converged ✅")
    #     break
    prev_ari = ari


Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


🔄 Iteration 1/9


/home/keerthana/anaconda3/envs/scmodal_scviva/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=95` in the `DataLoader` to improve performance.


Training:   0%|          | 0/100 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/keerthana/anaconda3/envs/scmodal_scviva/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=95` in the `DataLoader` to improve performance.


...

/home/keerthana/anaconda3/envs/scmodal_scviva/lib/python3.10/site-packages/scvi/external/scviva/_model.py:188: UserWarning: This dataset has some empty cells, this might fail inference.Data should be filtered with `scanpy.pp.filter_cells()`
  library_log_means, library_log_vars = _init_library_size(
Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/keerthana/anaconda3/envs/scmodal_scviva/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_worke

Training:   0%|          | 0/50 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


✅ Updated alpha and eta embeddings in the model (weights preserved).
Begining time:  Fri Nov  7 06:52:54 2025
🧩 Loading checkpoint with shape-safe mapping...
⚠️ Skipping optimizer_G load due to mismatch: local variable 'optimizer_G' referenced before assignment
⚠️ Skipping optimizer_D load due to mismatch: local variable 'optimizer_D' referenced before assignment
step 0, loss_D=0.608858, loss_GAN=-0.534559, loss_AE=21.719473, loss_Geo=-17.380024, loss_LA=0.294955, loss_MNN=0.693340
Ending time:  Fri Nov  7 06:52:59 2025
Training takes 5.10 seconds
Begining time:  Fri Nov  7 06:52:59 2025
Ending time:  Fri Nov  7 06:53:00 2025
Evaluating takes 0.15 seconds
Iteration ARI: 0.5359507195645788


Iteration ARI: 0.5914843154828876
🔄 Iteration 6/9
Epoch 43/100:  42%|████▏     | 42/100 [08:24<11:37, 12.03s/it, v_num=1, train_loss_step=174, train_loss_epoch=167]

In [16]:
np.unique(nichevae.adata.obs['imputed_cell_type'])

array(['B-cells', 'CAFs', 'Cancer Epithelial', 'Endothelial', 'Myeloid',
       'Normal Epithelial', 'PVL', 'Plasmablasts', 'T-cells'],
      dtype=object)

In [17]:
aris

[0.5415494193992968,
 0.5223655839919125,
 0.5451893292263683,
 0.5670703912204677,
 0.46030966632992215,
 0.4970313726203992,
 0.4843882349841435,
 0.5892611044623122,
 0.5359507195645788]